# Classificação de Observações Baseada em Similaridade

## Resumo

O projeto proposto visa desenvolver um sistema de classificação de observações que não possuem uma classe definida, com base em características e padrões identificados em um conjunto de dados pré-estabelecido. Em vez de atribuir rótulos de classe predefinidos, o sistema utilizará algoritmos de aprendizado de máquina para identificar grupos ou clusters de observações com características semelhantes, permitindo uma classificação baseada em similaridade.

## Objetivo

* Desenvolver e treinar modelos de machine learning para identificar padrões e similaridades em um conjunto de dados fornecido.
* Implementar algoritmos de clustering, como K-Means, DBSCAN, ou Hierarchical Clustering, para agrupar observações com base em suas características.
* Avaliar a eficácia dos modelos desenvolvidos através de métricas de avaliação de clusterização, como Silhouette Score, Calinski-Harabasz Index, ou Davies-Bouldin Index.
* Criar uma interface de usuário intuitiva para permitir a entrada de novas observações e classificá-las com base nos clusters identificados.

## Metodologia
* **Pré-processamento de Dados:** Limpar e preparar os dados, lidar com valores ausentes, normalizar ou padronizar características, se necessário.
* **Seleção de Algoritmo:** Selecionar o algoritmo de clustering mais apropriado com base nas características dos dados e nos requisitos do projeto.
* **Treinamento do Modelo:** Treinar o modelo de clustering nos dados pré-processados.
* **Avaliação do Modelo:** Avaliar a qualidade dos clusters formados usando métricas apropriadas.
* **Implementação do Sistema:** Desenvolver uma interface de usuário para permitir a entrada de novas observações e classificá-las com base nos clusters identificados.
* **Teste e Refinamento:** Testar o sistema com diferentes conjuntos de dados e refinar os algoritmos, se necessário, para melhorar a precisão e a eficácia da classificação.

# Análise exploratória

In [79]:
columns = "Unnamed: 0, ORIGEM_LANCAMENTO, DOCUMENTO_SAP, ITEM_DOCUMENTO_SAP, OPERACAO_REFERENCIA, CLASSE_CUSTO, MATERIAL_BASICO, DS_CLASSE_CUSTO, REF_ITEM_DOCUMENTO_FI, CD_EQUIPTO10, DS_EQUIPTO, LOCAL_EQUIPTO, DS_CLASSE_CUSTO_EQUIPTO, PARCELA_EQUIPTO, PARCELA_CUSTEIO, VALOR_BRL".split(", ")

In [92]:
df = pd.read_csv('data.csv', index_col='Unnamed: 0', usecols=columns)
df

,ORIGEM_LANCAMENTO,DOCUMENTO_SAP,ITEM_DOCUMENTO_SAP,OPERACAO_REFERENCIA,CLASSE_CUSTO,DS_CLASSE_CUSTO,MATERIAL_BASICO,REF_ITEM_DOCUMENTO_FI,CD_EQUIPTO10,DS_EQUIPTO,LOCAL_EQUIPTO,DS_CLASSE_CUSTO_EQUIPTO,PARCELA_EQUIPTO,PARCELA_CUSTEIO,VALOR_BRL
0,RATEIO_SERVICO,1026199426,54,ZE0063,5126020141,Escavadeira Sb Estei Alugado,NaN,54,3140000705,ORSINI,DWPENH,Escavadeira Sobre Esteiras,OUTROS,OPERAÇÃO MECÂNICA,-0.01
1,RATEIO_SERVICO,1026199426,56,ZE0063,5126020165,Motonivel C Escarifi Alugado,NaN,56,5850330000,BRILHANTINA,DWPENH,Motoniveladora c/Escarificador,OUTROS,OPERAÇÃO MECÂNICA,-0.01
2,RATEIO_SERVICO,1026199426,58,ZE0063,5126020193,Trator Pneus Agrico Alugado,NaN,58,5852030000,TICO TICO,DWPENH,Trator Pneus Agricola,OUTROS,OPERAÇÃO MECÂNICA,-0.01
3,RATEIO_SERVICO,1026199426,60,ZE0063,5126020177,Rolo Comp Rolo E Pne Alugado,NaN,60,5857850000,GUAVIROVEIRA,DWPENH,Rolo Compactador Rolo E Pneus,OUTROS,OPERAÇÃO MECÂNICA,-0.01
4,RATEIO_SERVICO,1026199426,62,ZE0063,5126020165,Motonivel C Escarifi Alugado,NaN,62,5859980000,MARAJO,DWPENH,Motoniveladora c/Escarificador,OUTROS,OPERAÇÃO MECÂNICA,-0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605434,PEP_EQ,1026172177,27,BKPF,5121001135,Peças para Equipamento,NaN,27,5861880000,PARDAL,DWWEPI,Trator Esteira c/Escarificador,OUTROS,MATERIAIS,1065.31
605435,PEP_EQ,314093479,2,AUAK,5121001027,Combustiveis,NaN,0,3230006426,WPENH AL 0409,DWPENH,Cam Tanque Pipa 6X4 Alugado,COMBUSTÍVEIS,MATERIAIS,962.91
605436,PEP_EQ,314030283,2,AUAK,5121001001,Abrasivos,NaN,0,5739220000,SIMPLICIO,DARIPU,Jumbo 3 Braç + Cesta,OUTROS,MATERIAIS,52.94
605437,PEP_EQ,1026093927,3,MKPF,5121001027,Combustiveis,01 - COMBUSTIVEL,6,3220004566,WPMG2 - AL 1319,DWPMG2,Veiculo Jeep Alugado,COMBUSTÍVEIS,MATERIAIS,143.33


## Observações
* CLASSE_CUSTO e DS_CLASSE_CUSTO ambos campos  representam o mesmo dado, decidir qual escolher a frente

In [111]:
def tratando_valores(material_basico):
    valores = {
        '01 - COMBUSTIVEL': 'COMBUSTÍVEIS',
        '03 - DESGASTE': 'DESGASTE',
        '05 - LUBRIFICANTE': 'LUBRIFICANTES',
        '06 - MATERIAL RODANTE': 'MATERIAL RODANTE',
        '07 - PECAS': 'PEÇAS',
        '08 - PNEU': 'PNEU',
        '99 - IMPOSTOS / TAXAS': 'TAXAS / IMPOSTOS'
    }
    return valores.get(material_basico, 'OUTROS')

df = df[df['MATERIAL_BASICO'].notna()][['MATERIAL_BASICO', 'CLASSE_CUSTO', 'DS_CLASSE_CUSTO', 'DS_CLASSE_CUSTO_EQUIPTO', 'PARCELA_EQUIPTO', 'PARCELA_CUSTEIO']]
df['CLASSE_EQUIPTO'] = df['MATERIAL_BASICO'].apply(tratando_valores)

In [112]:
df

,MATERIAL_BASICO,CLASSE_CUSTO,DS_CLASSE_CUSTO,DS_CLASSE_CUSTO_EQUIPTO,PARCELA_EQUIPTO,PARCELA_CUSTEIO,CLASSE_EQUIPTO
9395,07 - PECAS,5121001145,Peças para Veiculos Leves,Veiculo Ambulancia,PEÇAS,MATERIAIS,PEÇAS
9396,07 - PECAS,5121001145,Peças para Veiculos Leves,Veiculo Ambulancia,PEÇAS,MATERIAIS,PEÇAS
9397,07 - PECAS,5121001145,Peças para Veiculos Leves,Veiculo Ambulancia,PEÇAS,MATERIAIS,PEÇAS
9398,07 - PECAS,5121001135,Peças para Equipamento,Retro Escavadeira 4X4,PEÇAS,MATERIAIS,PEÇAS
9399,07 - PECAS,5121001145,Peças para Veiculos Leves,Veiculo Ambulancia,PEÇAS,MATERIAIS,PEÇAS
...,...,...,...,...,...,...,...
605286,00 - CONSUMO,5121001027,Combustiveis,Veiculo Automovel Alugado,COMBUSTÍVEIS,MATERIAIS,OUTROS
605352,01 - COMBUSTIVEL,5121001027,Combustiveis,Veiculo Pick Up Alugado,COMBUSTÍVEIS,MATERIAIS,COMBUSTÍVEIS
605369,07 - PECAS,5121001135,Peças para Equipamento,Escavadeira Sobre Esteiras,PEÇAS,MATERIAIS,PEÇAS
605415,01 - COMBUSTIVEL,5121001027,Combustiveis,Veiculo Jeep Alugado,COMBUSTÍVEIS,MATERIAIS,COMBUSTÍVEIS


In [104]:
df[df['MATERIAL_BASICO']=='00 - CONSUMO'].sample(30)

,ORIGEM_LANCAMENTO,DOCUMENTO_SAP,ITEM_DOCUMENTO_SAP,OPERACAO_REFERENCIA,CLASSE_CUSTO,DS_CLASSE_CUSTO,MATERIAL_BASICO,REF_ITEM_DOCUMENTO_FI,CD_EQUIPTO10,DS_EQUIPTO,LOCAL_EQUIPTO,DS_CLASSE_CUSTO_EQUIPTO,PARCELA_EQUIPTO,PARCELA_CUSTEIO,VALOR_BRL
203200,ORDEM_MANUT,1025247069,9,MKPF,5121001054,Elementos de fixação,00 - CONSUMO,18,3130000196,CHICO BALA,DRMIN0,Cam Basculante 6X4 Rocha,OUTROS,MATERIAIS,303.00
209409,ORDEM_MANUT,1025396484,9,MKPF,5121001054,Elementos de fixação,00 - CONSUMO,18,3280000853,WNOAR AL 229,DWNOAR,Plataforma Aerea Telescopica Ate-20 m,OUTROS,MATERIAIS,42.50
101641,ORDEM_MANUT,1025899780,15,MKPF,5121001136,Equipamento/Material Solda,00 - CONSUMO,30,3130000192,TAPICURU,DRMIN0,Cam Basculante 6X4 Rocha,OUTROS,MATERIAIS,150.00
38129,ORDEM_MANUT,1024810025,4,RMRP,5121001055,Embalagens,00 - CONSUMO,5,5850250000,BANANEIRINHA,DWSJAN,Motoniveladora c/Escarificador,OUTROS,MATERIAIS,25.80
224036,ORDEM_MANUT,1025546655,1,MKPF,5121001053,"Mangueiras, Mangotes e Acessorios",00 - CONSUMO,2,3240005591,WWEPI AL 210,DWWEPI,Trator Esteira C Esc Alugado,OUTROS,MATERIAIS,817.00
209537,ORDEM_MANUT,1025397175,7,MKPF,5121001054,Elementos de fixação,00 - CONSUMO,14,3130000466,VISCONDE,DRMIN0,Cam Basculante 6X4 Rocha,OUTROS,MATERIAIS,6.30
205146,ORDEM_MANUT,1025257479,2,MKPF,5121001054,Elementos de fixação,00 - CONSUMO,4,5846810000,ZEFIRANTES,DWWEPI,Cam Tanque Pipa 6X4,OUTROS,MATERIAIS,16.00
202852,ORDEM_MANUT,1025231929,4,MKPF,5121001053,"Mangueiras, Mangotes e Acessorios",00 - CONSUMO,8,3240004699,ARIPU AL 121,DARIPU,Retro Escavadeir 4X4 Alugado,OUTROS,MATERIAIS,27.60
546037,PEP_EQ,1026387879,5,MKPF,5121001029,Graxa,00 - CONSUMO,10,3240005665,WFLRV AL 2011 EH047,DWFLRV,Escavadeira Sb Estei Alugado,OUTROS,MATERIAIS,304.53
587258,PEP_EQ,1025745086,13,MKPF,5121001027,Combustiveis,00 - CONSUMO,26,3220003446,WSFUT AL 3014,DWSFUT,Veiculo Automovel Alugado,COMBUSTÍVEIS,MATERIAIS,-280.47


In [78]:
df_y = df[df['MATERIAL_BASICO'].isna()][['MATERIAL_BASICO', 'DS_CLASSE_CUSTO', 'DS_CLASSE_CUSTO_EQUIPTO', 'PARCELA_EQUIPTO', 'PARCELA_CUSTEIO']]
df_y[df_y['PARCELA_EQUIPTO'] == 'OUTROS']

,MATERIAL_BASICO,DS_CLASSE_CUSTO,DS_CLASSE_CUSTO_EQUIPTO,PARCELA_EQUIPTO,PARCELA_CUSTEIO
0,NaN,Escavadeira Sb Estei Alugado,Escavadeira Sobre Esteiras,OUTROS,OPERAÇÃO MECÂNICA
1,NaN,Motonivel C Escarifi Alugado,Motoniveladora c/Escarificador,OUTROS,OPERAÇÃO MECÂNICA
2,NaN,Trator Pneus Agrico Alugado,Trator Pneus Agricola,OUTROS,OPERAÇÃO MECÂNICA
3,NaN,Rolo Comp Rolo E Pne Alugado,Rolo Compactador Rolo E Pneus,OUTROS,OPERAÇÃO MECÂNICA
4,NaN,Motonivel C Escarifi Alugado,Motoniveladora c/Escarificador,OUTROS,OPERAÇÃO MECÂNICA
...,...,...,...,...,...
605431,NaN,Equipamentos/Implementos/Acessorios,Cam Tanque Pipa 6X4,OUTROS,MATERIAIS
605432,NaN,Aluguel de veículos,Veiculo Pick Up Alugado,OUTROS,OUTROS
605433,NaN,Cam Bascul 6X4 Terra Alugado,Cam Bascul 6X4 Terra Alugado,OUTROS,OPERAÇÃO MECÂNICA
605434,NaN,Peças para Equipamento,Trator Esteira c/Escarificador,OUTROS,MATERIAIS


In [17]:
df['ORIGEM_LANCAMENTO'].unique()

array(['RATEIO_SERVICO', 'ORDEM_MANUT', 'ORDEM_APROP', 'PEP_EQ'],
      dtype=object)

In [18]:
df['OPERACAO_REFERENCIA'].unique()

array(['ZE0063', 'FB08', 'MKPF', 'RMRP', 'AUAK', 'BKPF', 'FB50', 'FBR2',
       nan, 'BKPFF', 'AMDP', 'AMBU', 'REACI', 'CAJO', 'COBK', 'ACE',
       'VBRK'], dtype=object)

In [20]:
df['DS_CLASSE_CUSTO'].unique()

array(['Escavadeira Sb Estei Alugado', 'Motonivel C Escarifi Alugado',
       'Trator Pneus Agrico Alugado', 'Rolo Comp Rolo E Pne Alugado',
       'Veiculo Pick Up Alugado', 'Veiculo Utilitario Alugado',
       'Veiculo Automovel Alugado', 'Veiculo Furgao Carga Alugado',
       'Cam Guindauto 6X4 Alugado', 'Cam Tanque Pipa 4X2 Alugado',
       'Cam Tran Pessoal 4X2 Alugado', 'Cam Carroceria 4X2 Alugado',
       'Cam Tanque Pipa 6X4 Alugado', 'Cam Betoneira 6X4 Alugado',
       'Cam Bascul 6X4 Rocha Alugado', 'Retro Escavadeir 4X4 Alugado',
       'Cam Guindauto 4X2 Alugado',
       'Cam Plat Pant Fora Estrada Artic Alugado',
       'Cam Espargidor 4X2 Alugado', 'Cam Plataforma 6X4 Alugado',
       'Trator Esteira C Esc Alugado', 'Rolo Comp Pe Carneir Alugado',
       'Grupo Gerador Estac Alugado', 'Carreg Sobre Esteira Alugado',
       'Compressor Rebocavel Alugado', 'Manipulador Telescop Alugado',
       'Retro Escavadeira 4X2 81-90 cv', 'Cavalo Mecanico 6X4 Alugado',
       'Veiculo

In [28]:
len(df['DS_CLASSE_CUSTO'].value_counts()[df['DS_CLASSE_CUSTO'].value_counts() > 1000])

68

In [33]:
list(df['DS_CLASSE_CUSTO'].value_counts()[df['DS_CLASSE_CUSTO'].value_counts() > 1000].index)

['Combustiveis',
 'Depreciação e Amortização de Ativo Imobi',
 'Peças para Veiculos Leves',
 'Peças para Equipamento',
 'Conservação e Reparação Equipamentos',
 'Cam Bascul 6X4 Terra Alugado',
 'Aluguel de equipamentos',
 'Oleo Lubrificante',
 'Escavadeira Sb Estei Alugado',
 'Cam Bascul 6X4 Rocha Alugado',
 'Material Electrico/Electronico',
 'Aluguel de veículos',
 'Materiais',
 'Veiculo Pick Up Alugado',
 'Cam Tanque Pipa 6X4 Alugado',
 'Cam Guindauto 6X4 Alugado',
 'Outros Trabalhos Especializados',
 'Retro Escavadeir 4X4 Alugado',
 'Trator Esteira C Esc Alugado',
 'Outros Serviços',
 'Equipamento de Transporte',
 'Motonivel C Escarifi Alugado',
 'Embalagens',
 'Cam Betoneira 6X4 Alugado',
 'Equipamento Básico',
 'Graxa',
 'Outros Gastos com Equipamentos',
 'Mangueiras, Mangotes e Acessorios',
 'Carreg Sobre Pneus Alugado',
 'Cam Bascul 8X4 Rocha Alugado',
 'Veiculo Automovel Alugado',
 'Manipulador Telescop Alugado',
 'Subempreiteiros Montagem Mecânica',
 'Rolo Comp Rolo E Pne Alug

In [34]:
df['PARCELA_EQUIPTO'].unique()

array(['OUTROS', 'DEPRECIAÇÃO', 'PEÇAS', 'MATERIAL DE DESGASTE',
       'MATERIAL RODANTE', 'LUBRIFICANTES', 'TAXAS / IMPOSTOS',
       'COMBUSTÍVEIS', 'ABATE', 'MÃO-DE-OBRA'], dtype=object)

In [35]:
df['PARCELA_CUSTEIO'].unique()

array(['OPERAÇÃO MECÂNICA', 'OUTROS', 'MATERIAIS', 'MÃO-DE-OBRA'],
      dtype=object)

In [63]:
len(df['MATERIAL_BASICO'].isna())#.value_counts()

605439

In [54]:
len(df[df['MATERIAL_BASICO']=='07 - PECAS'])

28571

In [57]:
df[df['PARCELA_EQUIPTO']=='COMBUSTÍVEIS'][['MATERIAL_BASICO', 'DS_CLASSE_CUSTO']]#.unique()

,MATERIAL_BASICO,DS_CLASSE_CUSTO
39855,01 - COMBUSTIVEL,Combustiveis
40030,01 - COMBUSTIVEL,Combustiveis
40265,NaN,Combustiveis
40270,NaN,Combustiveis
74380,07 - PECAS,Combustiveis
...,...,...
605412,NaN,Combustiveis
605415,01 - COMBUSTIVEL,Combustiveis
605425,NaN,Combustiveis
605435,NaN,Combustiveis


In [46]:
df[df['DS_CLASSE_CUSTO']=='Diesel/Gasóleo']['PARCELA_EQUIPTO'].unique()

array(['COMBUSTÍVEIS'], dtype=object)

In [42]:
df[df['DS_CLASSE_CUSTO']=='Peças para Equipamento']['PARCELA_EQUIPTO'].unique()

array(['PEÇAS', 'OUTROS', 'MATERIAL DE DESGASTE', 'MATERIAL RODANTE',
       'LUBRIFICANTES'], dtype=object)

In [13]:
df.dtypes

ORIGEM_LANCAMENTO           object
DOCUMENTO_SAP                int64
ITEM_DOCUMENTO_SAP           int64
OPERACAO_REFERENCIA         object
CD_ELEMENTO_PEP             object
CLASSE_CUSTO                 int64
DS_CLASSE_CUSTO             object
REF_ITEM_DOCUMENTO_FI        int64
CD_EQUIPTO10                 int64
DS_EQUIPTO                  object
LOCAL_EQUIPTO               object
DS_CLASSE_CUSTO_EQUIPTO     object
PARCELA_EQUIPTO             object
PARCELA_CUSTEIO             object
VALOR_BRL                  float64
dtype: object

In [15]:
df.describe()

,DOCUMENTO_SAP,ITEM_DOCUMENTO_SAP,CLASSE_CUSTO,REF_ITEM_DOCUMENTO_FI,CD_EQUIPTO10,VALOR_BRL
count,6.054390e+05,605439.000000,6.054390e+05,605439.000000,6.054390e+05,6.054390e+05
mean,8.364233e+08,97.023931,5.168941e+09,91.390857,3.700543e+09,1.177275e+01
std,3.141951e+08,182.866089,1.086295e+08,171.552485,1.115956e+09,2.112515e+06
min,3.139841e+08,1.000000,9.100040e+05,0.000000,1.576630e+09,-2.279396e+08
25%,3.142667e+08,1.000000,5.121001e+09,0.000000,3.140000e+09,-2.437950e+02
50%,1.025046e+09,5.000000,5.126020e+09,5.000000,3.230004e+09,5.850000e+01
75%,1.025729e+09,94.000000,5.126090e+09,92.000000,3.240005e+09,7.459350e+02
max,1.026547e+09,900.000000,9.160000e+09,900.000000,9.099120e+09,2.279396e+08
